In [ ]:
# item, rating, timestamp, status, completion, user, position
const wordtype = Tuple{Int32,Float32,Float32,Int32,Float32,Int32,Int32}

function get_wordtype_index(field::Symbol)
    if field == :item
        return 1
    elseif field == :rating
        return 2
    elseif field == :timestamp
        return 3
    elseif field == :status
        return 4
    elseif field == :completion
        return 5
    elseif field == :user
        return 6
    elseif field == :position
        return 7
    else
        @assert false
    end
end

extract(word, field::Symbol) = word[get_wordtype_index(field)]

function replace(word, fieldname::Symbol, field)
    pos = get_wordtype_index(fieldname)
    (word[1:pos-1]..., field, word[pos+1:end]...)
end

function encode_word(
    item,
    rating,
    timestamp,
    status,
    completion,
    user,
    position,
)
    word =
        (item, rating, timestamp, status, completion, user, position)
    convert(wordtype, word)
end

is_ptw(word::wordtype) = extract(word, :status) == 1;

In [ ]:
function encode_word(item, rating, timestamp, status, completion, user, position)
    word = (item, rating, timestamp, status, completion, user, position)
    convert(wordtype, word)
end

function get_training_data(task, include_ptw, cls_tokens; show_progress_bar = false)
    function get_df(task, content)
        df = get_raw_split("training", task, content)
        if content != "explicit"
            df.rating .= 11
        end
        df
    end

    contents = ["explicit", "implicit"]
    if include_ptw
        push!(contents, "ptw")
    end
    sentences = Dict{Int32,Vector{wordtype}}()
    df = reduce(cat, [get_df(task, content) for content in contents])
    order = sortperm(df.timestamp)
    p = ProgressMeter.Progress(length(order); enabled = show_progress_bar, showspeed = true)
    for idx = 1:length(order)
        i = order[idx]
        if df.user[i] ∉ keys(sentences)
            sentences[df.user[i]] = [replace(cls_tokens, :user, df.user[i])]
        end
        word = encode_word(
            df.item[i],
            df.rating[i],
            df.timestamp[i],
            df.status[i],
            df.completion[i],
            df.user[i],
            length(sentences[df.user[i]]),
        )
        push!(sentences[df.user[i]], word)
        ProgressMeter.next!(p)
    end
    ProgressMeter.finish!(p)
    sentences
end;

In [ ]:
function subset_sentence(sentence, max_seq_length; recent, rng)
    if length(sentence) > max_seq_length
        if recent
            # keep the rightmost entries
            idx = length(sentence) - max_seq_length + 1
        else
            # take a random contiguous subset            
            idx = rand(rng, 1:length(sentence)-max_seq_length+1)
        end
        sentence = sentence[idx:idx+max_seq_length-1]
    end
    sentence
end;

In [ ]:
function pad_sentence(sentence, max_seq_length, max_position, pad_tokens, cls_tokens; rng)
    outputs = fill.(pad_tokens, max_seq_length)
    sentence = subset_sentence(sentence, max_seq_length; recent = false, rng = rng)
    for i = 1:length(sentence)
        for j = 1:length(outputs)
            if j == get_wordtype_index(:position) && extract(sentence[i], :item) != extract(cls_tokens, :item)
                p = (sentence[i][j] % max_position)
                if p == 0
                    p = max_position
                end
                outputs[j][i] = p
            else
                outputs[j][i] = sentence[i][j]
            end
        end
    end
    outputs
end;

In [ ]:
function get_token_ids(sentences, max_seq_length, max_position, pad_tokens, cls_tokens; rng)
    padded_sentences = [
        pad_sentence(x, max_seq_length, max_position, pad_tokens, cls_tokens; rng = rng) for x in sentences
    ]
    Tuple(hcat([x[i] for x in padded_sentences]...) for i = 1:length(pad_tokens))
end;